In [1]:
#!pip install import-ipynb

In [ ]:
#!pip install ImageHash

In [2]:
#!pip install pyautogui


In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# Monthly amount of employees of the last two years
def extract_employee_growth(driver, company_name):
    # Get the page
    #driver.get(f"https://www.linkedin.com/company/{company_name}/insights/")
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Adjust the selector as per the actual HTML structure
    svg_code = soup.find('g', class_='highcharts-series-group')

    # Initialize a dictionary for storing company data
    company_data = {'Company': company_name}

    # If SVG code was found, extract it
    if svg_code:
        svg_code = str(svg_code)

        # Regular expression to extract the employee counts from aria-label attributes
        pattern = r'aria-label="[^"]* (\d{1,3}(?:,\d{3})*) employees'

        # Find all matches
        employee_counts = re.findall(pattern, svg_code)

        # Clean up the numbers (remove commas)
        employee_counts = [int(count.replace(',', '')) for count in employee_counts]

        # Create a list of month labels from October 2022 to October 2024
        start_date = datetime(2022, 10, 1)
        
        # Generate month labels corresponding to the number of employee counts
        month_labels = []

        for i in range(len(employee_counts)):
            month_labels.append((start_date.month + i) % 12 or 12)  # Month in range 1-12
            year = start_date.year + (start_date.month + i - 1) // 12  # Calculate the correct year
            month_labels[-1] = f"{datetime(year, month_labels[-1], 1).strftime('%B %Y')}"

        # Add employee counts to the company data dictionary with month labels as keys
        for month, count in zip(month_labels, employee_counts):
            company_data[month] = count
    else:
        print(f"SVG code not found for {company_name}.")

    return company_data
"""
# Assuming you have a list of company names to loop through
company_names = ['tesla-motors', 'another-company', 'yet-another-company']  # Example company names
result_df = pd.DataFrame()  # Initialize an empty DataFrame

# Example usage
driver = webdriver.Chrome()  # Initialize your driver here

for company in company_names:
    company_data = extract_employee_growth(driver, company)
    # Convert the dictionary to a DataFrame
    temp_df = pd.DataFrame([company_data])  # Create a DataFrame from the dictionary
    # Concatenate the new DataFrame to the result_df
    result_df = pd.concat([result_df, temp_df], ignore_index=True)

# Clean up the driver
#driver.quit()

# Print the resulting DataFrame
print(result_df)
"""

"\n# Assuming you have a list of company names to loop through\ncompany_names = ['tesla-motors', 'another-company', 'yet-another-company']  # Example company names\nresult_df = pd.DataFrame()  # Initialize an empty DataFrame\n\n# Example usage\ndriver = webdriver.Chrome()  # Initialize your driver here\n\nfor company in company_names:\n    company_data = extract_employee_growth(driver, company)\n    # Convert the dictionary to a DataFrame\n    temp_df = pd.DataFrame([company_data])  # Create a DataFrame from the dictionary\n    # Concatenate the new DataFrame to the result_df\n    result_df = pd.concat([result_df, temp_df], ignore_index=True)\n\n# Clean up the driver\n#driver.quit()\n\n# Print the resulting DataFrame\nprint(result_df)\n"

In [5]:
#Amount of employees in the different positions
def employee_distribution(driver, company_name):
    # Get the page
    #driver.get(f"https://www.linkedin.com/company/{company_name}/insights/")
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find the specific div containing the target tbody
    div_growth_table = soup.find('div', class_='org-function-growth-table')
    
    # Prepare a dictionary to store the extracted data for this company
    company_data = {'Company': company_name}
    
    # Check if the div was found
    if div_growth_table:
        tbody = div_growth_table.find('tbody')
        
        # Loop through each row in the tbody
        for row in tbody.find_all('tr'):
            # Extracting the area name
            area_cell = row.find('td', headers="org-function-growth-table__a11y-functions-function")
            num_employees_cell = row.find('td', headers="org-function-growth-table__a11y-functions-num-employees")
            
            # Check if area and number of employees were found
            if area_cell and num_employees_cell:
                area = area_cell.text.strip()
                num_employees = int(num_employees_cell.text.strip())
                company_data[area] = num_employees  # Add to dictionary
            else:
                print("Some data not found in this row.")
    else:
        print("The specified div was not found in the response.")
    
    return company_data
"""
# Assuming you have a list of company names to loop through
company_names = ['tesla-motors', 'another-company', 'yet-another-company']  # Example company names
result_df = pd.DataFrame()  # Initialize an empty DataFrame

# Example usage
driver = webdriver.Chrome()  # Initialize your driver here

for company in company_names:
    company_data = employee_distribution(driver, company)
    # Convert the dictionary to a DataFrame and append to the result_df
    result_df = result_df.append(company_data, ignore_index=True)

# Clean up the driver
#driver.quit()

# Print the resulting DataFrame
print(result_df)
"""

"\n# Assuming you have a list of company names to loop through\ncompany_names = ['tesla-motors', 'another-company', 'yet-another-company']  # Example company names\nresult_df = pd.DataFrame()  # Initialize an empty DataFrame\n\n# Example usage\ndriver = webdriver.Chrome()  # Initialize your driver here\n\nfor company in company_names:\n    company_data = employee_distribution(driver, company)\n    # Convert the dictionary to a DataFrame and append to the result_df\n    result_df = result_df.append(company_data, ignore_index=True)\n\n# Clean up the driver\n#driver.quit()\n\n# Print the resulting DataFrame\nprint(result_df)\n"

In [5]:
# Put data into dataframes
def employee_counter(internal_id, company_name, employee_number_df, driver):
    try:
        # Extract employee growth data for the specified company
        company_size_data = extract_employee_growth(driver, company_name)
    
        # Add the internal_id to the company size data
        company_size_data['Internal ID'] = internal_id  # Add internal_id to the data dictionary
    
        # Convert the updated dictionary to a DataFrame
        temp_size_df = pd.DataFrame([company_size_data])  # Create a DataFrame from the dictionary
    
        # Move 'Internal ID' to the first column and rearrange DataFrame columns
        temp_size_df = temp_size_df[['Internal ID'] + [col for col in temp_size_df.columns if col != 'Internal ID']]
    
        # Concatenate the new DataFrame to the employee_number_df
        employee_number_df = pd.concat([employee_number_df, temp_size_df], ignore_index=True)
    except:
        pass
    
    # Return the updated DataFrame
    return employee_number_df

    """# Assuming you have a list of company names to loop through
    #company_names = ['tesla-motors', 'another-company', 'yet-another-company']  # Example company names
    #result_df = pd.DataFrame()  # Initialize an empty DataFrame
    
    # Example usage
    #driver = webdriver.Chrome()  # Initialize your driver here
    
    #for company in company_names:
    company_size_data = extract_employee_growth(driver, company_name)
    # Convert the dictionary to a DataFrame
    temp_size_df = pd.DataFrame([company_size_data])  # Create a DataFrame from the dictionary
    # Concatenate the new DataFrame to the result_df
    employee_number_df = pd.concat([employee_number_df, temp_size_df], ignore_index=True)
    
    # Clean up the driver
    #driver.quit()
    
    # Print the resulting DataFrame
    return employee_number_df"""

def diversity(internal_id, company_name, employee_diversity_df, driver):
    try:
        # Extract employee diversity data for the specified company
        company_diversity_data = employee_distribution(driver, company_name)
    
        # Add the internal_id to the company diversity data
        company_diversity_data['Internal ID'] = internal_id  # Add internal_id to the data dictionary
    
        # Convert the updated dictionary to a DataFrame
        temp_diversity_df = pd.DataFrame([company_diversity_data])  # Create a DataFrame from the dictionary
    
        # Move 'Internal ID' to the first column and rearrange DataFrame columns
        temp_diversity_df = temp_diversity_df[['Internal ID'] + [col for col in temp_diversity_df.columns if col != 'Internal ID']]
    
        # Concatenate the new DataFrame to the employee_diversity_df
        employee_diversity_df = pd.concat([employee_diversity_df, temp_diversity_df], ignore_index=True)
    except:
        pass
    # Return the updated DataFrame
    return employee_diversity_df

    """#driver = webdriver.Chrome()  # Initialize your driver here
    company_diversity_data = employee_distribution(driver, company_name)
    # Convert the dictionary to a DataFrame
    temp_diversity_df = pd.DataFrame([company_diversity_data])  # Create a DataFrame from the dictionary
    # Concatenate the new DataFrame to the result_df
    employee_diversity_df = pd.concat([employee_diversity_df, temp_diversity_df], ignore_index=True)
    
    # Clean up the driver
    #driver.quit()
    
    # Print the resulting DataFrame
    return employee_diversity_df"""


In [3]:
def To_Insights(driver):
    # CSS selector to find the Insights button
    try:
        link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Insights"))  # Example XPATH"
        )
        
        # Click the button
        link.click()

    except Exception as e:
        print("error")
    """
    css_selector = "li.org-page-navigation__item.m0.mr1 a.ember-view.pv3.ph4.t-16.t-bold.t-black--light.org-page-navigation__item-anchor"
    
    
    try:
        # Wait for the element to be clickable
        insights_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
        )
        # Click the button
        insights_button.click()
        #print("Clicked the Insights button.")
    except Exception as e:
        print("error Clicking the Insights button")
        """